**Вопросы для повторения:**

* Что такое функтор? Что такое лямбда?

* Что здесь происходит?

```c++
auto make_age_less_filter(int x)
{
    return [&x](const Person& person){ return person.age < x; };
}

std::copy_if(people.begin(),
             people.end(),
             make_age_less_filter(33),
             std::ostream_iterator<Person>(std::cout, "\n"));
```

* Что здесь происходит? Всё ли в порядке?

```c++
unsigned compare_count = 0;
std::sort(v.begin(), v.end(),
          [&](const auto& lhs, const auto& rhs){
              ++compare_count;
              return lhs < rhs;
          });
```

* Понять что происходит, исправить ошибки и упростить:

```c++
struct Person
{
  std::string name;
  int age;
};

int function(std::vector<Persion> people)
{
  auto tmp = people;
  
  std::remove_if(tmp.begin(), tmp.end(), [](const Person& p){
    return p.age < 33;
  });

  std::sort(tmp.begin(), tmp.end(), [](const Person& lhs, const Person& rhs){
    reutrn lhs.age < rhs.age;
  });

  auto it = std::find_if(tmp.rbegin(), tmp.rend(), [](const Person& p){
    return p.name == "Fedor";
  });

  return it->age;
}
```

* Понять что происходит, исправить ошибки и упростить:

```c++
std::set<int, int> function(const std::vector<Person>& people)
{
  std::map<int, std::vector<std::string>> age_to_names;
  for (const auto& p : people)
    age_to_names[p.age].push_back(p.name);

  for (auto& age_and_names : age_to_names)
  {
    auto names = age_and_names.second;
    std::sort(names.begin(), names.end());
  }

  std::set<int, int> rv;
  for (auto& age_and_names : age_to_names)
    rv[age_and_names.first] = age_and_names.second.size();
    
  return rv;
}
```

* Понять что происходит, исправить ошибки и упростить:

```c++

std::set<int, int> function(const std::vector<Person>& people)
{
  std::map<int, std::vector<std::string>> age_to_names;
  for (const auto& p : people)
    age_to_names[p.age].push_back(p.name);

  for (auto& age_and_names : age_to_names)
  {
    auto names = age_and_names.second;
    std::sort(names.begin(), names.end());
  }

  std::set<int, int> rv;
  for (auto& age_and_names : age_to_names)
    rv[age_and_names.first] = age_and_names.second.size();
    
  return rv;
}
```



**Долг с прошлой лекции:**
* Если вы пишете алгоритм, и вам кажется, что он достаточно общий, значит, алгоритм до вас уже кто-то реализовывал, загляните в STL, скорее всего, он там уже есть.
* [toupper && unsigned chars](https://en.cppreference.com/w/cpp/string/byte/toupper)
* Примеры применения immediately call lambdas:

Вариант 1: inplace инициализация полей класса

```c++
class NetworkSystem
{
private:
    NetworkConfig config_;
    
public:
    NetworkSystem(const std::string& config_filename)
        : config_([](){
            const auto json = read_json(config_filename);
            
            NetworkConfig cfg;
            cfg.http_timeout = json['http']['timeout'];
            cfg.retry_count = json['retry_count'];
            return cfg;
          }())
    {
    }
            
   ...    
};
```


Вариант 2: inplace инициализация static-переменных

```c++
log4cplus::Logger& getDefaultLogger()
{
  static log4cplus::Logger s_log = [] {
    auto log = log4cplus::Logger::getInstance(L"default");
    // setup log properties here
    return log;
  }();

  return s_log;
}
```


Вариант 3:

```c++
void process_logs(const std::string& logs_filename, const Date& min_date)
{
    // initialize const records,
    // cleanup init harness on completion
    const auto log_recods = [&](){
        std::ifstream ifs(logs_filename);
        if (!ifs)
            throw std::runtime_error("filed to open log file: " + logs_filename);
        
        std::vector<LogRecord> records{std::istream_iterator(ifs),
                                       std::istream_iterator()};
        boost::remove_erase_if(records,
                               [&](const auto& rec) { return rec.date < min_date; });
        return records;
    }();
    
    // further const records processing
    ...
}
```
